In [2]:
import pandas as pd
%matplotlib inlin

We first want to develop an understanding of our current customer base. Let's look at what percentage of rides are taken by subscribers and what percentage are taken by either 24-hour or 3-day customers.

  bq query --use_legacy_sql=false '
SELECT
  SUBSCRIBER_TYPE,
  COUNT(*) AS RIDES,
  COUNT(*) / (
  SELECT
    COUNT(*)
  FROM
    `bigquery-public-data.san_francisco.bikeshare_trips`) AS PERC_RIDER_TYPE
FROM
  `bigquery-public-data.san_francisco.bikeshare_trips`
GROUP BY
  SUBSCRIBER_TYPE'

In [20]:
! bq query --use_legacy_sql=false 'SELECT SUBSCRIBER_TYPE, COUNT(*) AS RIDES, COUNT(*) / (SELECT COUNT(*) FROM `bigquery-public-data.san_francisco.bikeshare_trips`) AS PERC_RIDER_TYPE FROM `bigquery-public-data.san_francisco.bikeshare_trips` GROUP BY SUBSCRIBER_TYPE'

Waiting on bqjob_r4a146552148baaa2_000001640fad1740_1 ... (0s) Current status: DONE   
+-----------------+--------+---------------------+
| SUBSCRIBER_TYPE | RIDES  |   PERC_RIDER_TYPE   |
+-----------------+--------+---------------------+
| Customer        | 136809 | 0.13908328995738312 |
| Subscriber      | 846839 |  0.8609167100426168 |
+-----------------+--------+---------------------+


Here we can see that 14% of rides are taken by either 1- or 3-day members. There are two methods by which we can increase ridership: we can recruit completely new customers or we can convert 1- or 3-day members customers into subscribers. The former will be tricky for us to do given the data we are provided. Thus, we will focus on converting temporary riders, who are currently accounting for 14% of rides (136,809 trips), into full-time subscribers.

Let's find out whether trip duration varies based on rider type.

bq query --use_legacy_sql=false '
SELECT
  SUBSCRIBER_TYPE,
  COUNT(*) AS RIDES,
  COUNT(*) / (
  SELECT
    COUNT(*)
  FROM
    `bigquery-public-data.san_francisco.bikeshare_trips`) AS PERC_RIDER_TYPE,
  AVG(duration_sec) AS AVG_TRIP_DURATION
FROM
  `bigquery-public-data.san_francisco.bikeshare_trips`
GROUP BY
  SUBSCRIBER_TYPE'

In [21]:
! bq query --use_legacy_sql=false 'SELECT SUBSCRIBER_TYPE, COUNT(*) AS RIDES, COUNT(*) / (SELECT COUNT(*) FROM `bigquery-public-data.san_francisco.bikeshare_trips`) AS PERC_RIDER_TYPE, AVG(duration_sec) AS AVG_TRIP_DURATION FROM   `bigquery-public-data.san_francisco.bikeshare_trips` GROUP BY SUBSCRIBER_TYPE'

Waiting on bqjob_r74a26a6f3fe71d6a_000001640fbf67e6_1 ... (1s) Current status: DONE   
+-----------------+--------+---------------------+-------------------+
| SUBSCRIBER_TYPE | RIDES  |   PERC_RIDER_TYPE   | AVG_TRIP_DURATION |
+-----------------+--------+---------------------+-------------------+
| Customer        | 136809 | 0.13908328995738312 | 3718.785160333024 |
| Subscriber      | 846839 |  0.8609167100426168 | 582.7642397197222 |
+-----------------+--------+---------------------+-------------------+


Here we can see a staunch difference in average trip duration based on rider type: the average customer trip lasts 3718 seconds (just over an hour) while the average subscriber trip lasts 583 seconds (just under ten minutes). This makes sense intuitively as customers are likely to try “getting their money’s worth” out of any given transaction.

Next let's look at the most common trips grouped by time window.

bq query --use_legacy_sql=false '
SELECT
  COUNT(*) AS RIDES, START_STATION_NAME, END_STATION_NAME, AVG(DURATION_SEC) AVG_DURATION
  CASE
    WHEN EXTRACT(HOUR  FROM  START_DATE) BETWEEN 6 AND 12 THEN "MORNING"
    WHEN EXTRACT(HOUR  FROM  START_DATE) BETWEEN 12 AND 18 THEN "AFTERNOON"
    ELSE "NIGHT"
  END AS TIME_WINDOW
FROM
  `bigquery-public-data.san_francisco.bikeshare_trips`
GROUP BY
  TIME_WINDOW, START_STATION_NAME, END_STATION_NAME
ORDER BY COUNT(*) DESC
LIMIT 10'

In [68]:
! bq query --use_legacy_sql=false --format=csv 'SELECT COUNT(*) AS RIDES, START_STATION_NAME, END_STATION_NAME, AVG(DURATION_SEC) AVG_DURATION, CASE WHEN EXTRACT(HOUR  FROM  START_DATE) BETWEEN 6 AND 12 THEN "MORNING" WHEN EXTRACT(HOUR  FROM  START_DATE) BETWEEN 12 AND 18 THEN "AFTERNOON" ELSE "NIGHT" END AS TIME_WINDOW FROM `bigquery-public-data.san_francisco.bikeshare_trips` GROUP BY TIME_WINDOW, START_STATION_NAME, END_STATION_NAME ORDER BY COUNT(*) DESC LIMIT 10' > mostcommon.csv

Waiting on bqjob_r7ad867d33084de8e_00000164109996e3_1 ... (0s) Current status: DONE   

In [69]:
MC = pd.read_csv('mostcommon.csv')
MC

,RIDES,START_STATION_NAME,END_STATION_NAME,AVG_DURATION,TIME_WINDOW
0,5552,2nd at Townsend,Harry Bridges Plaza (Ferry Building),532.014950,AFTERNOON
1,5273,Harry Bridges Plaza (Ferry Building),2nd at Townsend,599.813579,MORNING
2,5063,Embarcadero at Folsom,San Francisco Caltrain (Townsend at 4th),670.056488,AFTERNOON
3,4993,Harry Bridges Plaza (Ferry Building),Embarcadero at Sansome,1056.502504,MORNING
4,4783,Embarcadero at Sansome,Steuart at Market,526.946895,AFTERNOON
5,4740,San Francisco Caltrain 2 (330 Townsend),Townsend at 7th,303.528692,MORNING
6,4561,Steuart at Market,2nd at Townsend,528.207849,MORNING
7,4515,San Francisco Caltrain (Townsend at 4th),Temporary Transbay Terminal (Howard at Beale),693.508970,MORNING
8,4160,Steuart at Market,San Francisco Caltrain (Townsend at 4th),736.295192,AFTERNOON
9,4143,Temporary Transbay Terminal (Howard at Beale),San Francisco Caltrain (Townsend at 4th),672.406951,AFTERNOON


The two most common trips are 2nd at Townsend to Harry Bridges Plaza during the afternoon and the inverse during the morning. This is a good example of a probable commuter trip.

Let's segment this further and look at the ten most common customer trips. 

bq query --use_legacy_sql=false --format=csv 'SELECT COUNT(*) AS RIDES, START_STATION_NAME, END_STATION_NAME, AVG(DURATION_SEC) AVG_DURATION, CASE WHEN EXTRACT(HOUR  FROM  START_DATE) BETWEEN 6 AND 12 THEN "MORNING" WHEN EXTRACT(HOUR  FROM  START_DATE) BETWEEN 12 AND 18 THEN "AFTERNOON" ELSE "NIGHT" END AS TIME_WINDOW FROM `bigquery-public-data.san_francisco.bikeshare_trips` WHERE SUBSCRIBER_TYPE = 'Customer' GROUP BY TIME_WINDOW, START_STATION_NAME, END_STATION_NAME ORDER BY COUNT(*) DESC LIMIT 10' > mostcommoncustomer.csv

In [73]:
! bq query --use_legacy_sql=false --format=csv 'SELECT COUNT(*) AS RIDES, START_STATION_NAME, END_STATION_NAME, AVG(DURATION_SEC) AVG_DURATION, CASE WHEN EXTRACT(HOUR  FROM  START_DATE) BETWEEN 6 AND 12 THEN "MORNING" WHEN EXTRACT(HOUR  FROM  START_DATE) BETWEEN 12 AND 18 THEN "AFTERNOON" ELSE "NIGHT" END AS TIME_WINDOW FROM `bigquery-public-data.san_francisco.bikeshare_trips` WHERE SUBSCRIBER_TYPE = "Customer" GROUP BY TIME_WINDOW, START_STATION_NAME, END_STATION_NAME ORDER BY COUNT(*) DESC LIMIT 10' > mostcommoncustomer.csv

Waiting on bqjob_rec98ae2fc1225d1_00000164109aca6e_1 ... (0s) Current status: DONE   

In [74]:
MCC = pd.read_csv('mostcommoncustomer.csv')
MCC

,RIDES,START_STATION_NAME,END_STATION_NAME,AVG_DURATION,TIME_WINDOW
0,1943,Harry Bridges Plaza (Ferry Building),Embarcadero at Sansome,2227.812661,AFTERNOON
1,1536,Embarcadero at Sansome,Embarcadero at Sansome,4511.128255,AFTERNOON
2,1505,Harry Bridges Plaza (Ferry Building),Embarcadero at Sansome,2502.603322,MORNING
3,1032,Harry Bridges Plaza (Ferry Building),Harry Bridges Plaza (Ferry Building),5819.851744,AFTERNOON
4,1002,Embarcadero at Sansome,Harry Bridges Plaza (Ferry Building),1439.026946,AFTERNOON
5,786,Embarcadero at Sansome,Embarcadero at Sansome,5624.424936,MORNING
6,762,Harry Bridges Plaza (Ferry Building),Harry Bridges Plaza (Ferry Building),8798.430446,MORNING
7,727,Embarcadero at Vallejo,Embarcadero at Sansome,2534.141678,AFTERNOON
8,594,University and Emerson,University and Emerson,8042.436027,AFTERNOON
9,528,Embarcadero at Vallejo,Embarcadero at Sansome,2666.198864,MORNING


This table yields several interesting findings. The first is that not surprisingly, the average duration of the most common customer trips is much greater than the average duration of all trips. All of the top ten most common customer trips last over 1439 seconds on average and several last more than two hours (7200 seconds). We saw earlier that the average subscriber trip lasts only 582 seconds. 

A second discovery is that five of the top ten most common customer trips begin and end at the same station. This, coupled with the length of the trips, leads us to believe that a large portion of customer trips can be described as joy rides.

Let's do the same thing for subscriber trips.

bq query --use_legacy_sql=false --format=csv 'SELECT COUNT(*) AS RIDES, START_STATION_NAME, END_STATION_NAME, AVG(DURATION_SEC) AVG_DURATION, CASE WHEN EXTRACT(HOUR  FROM  START_DATE) BETWEEN 6 AND 12 THEN "MORNING" WHEN EXTRACT(HOUR  FROM  START_DATE) BETWEEN 12 AND 18 THEN "AFTERNOON" ELSE "NIGHT" END AS TIME_WINDOW FROM `bigquery-public-data.san_francisco.bikeshare_trips` WHERE SUBSCRIBER_TYPE = 'Subscriber' GROUP BY TIME_WINDOW, START_STATION_NAME, END_STATION_NAME ORDER BY COUNT(*) DESC LIMIT 10' > mostcommonsubscriber.csv

In [76]:
! bq query --use_legacy_sql=false --format=csv 'SELECT COUNT(*) AS RIDES, START_STATION_NAME, END_STATION_NAME, AVG(DURATION_SEC) AVG_DURATION, CASE WHEN EXTRACT(HOUR  FROM  START_DATE) BETWEEN 6 AND 12 THEN "MORNING" WHEN EXTRACT(HOUR  FROM  START_DATE) BETWEEN 12 AND 18 THEN "AFTERNOON" ELSE "NIGHT" END AS TIME_WINDOW FROM `bigquery-public-data.san_francisco.bikeshare_trips` WHERE SUBSCRIBER_TYPE = "Subscriber" GROUP BY TIME_WINDOW, START_STATION_NAME, END_STATION_NAME ORDER BY COUNT(*) DESC LIMIT 10' > mostcommonsubscriber.csv

Waiting on bqjob_r75429942ec837fe5_0000016410a675bf_1 ... (1s) Current status: DONE   

In [77]:
MCS = pd.read_csv('mostcommonsubscriber.csv')
MCS

,RIDES,START_STATION_NAME,END_STATION_NAME,AVG_DURATION,TIME_WINDOW
0,5134,2nd at Townsend,Harry Bridges Plaza (Ferry Building),489.053759,AFTERNOON
1,5099,Harry Bridges Plaza (Ferry Building),2nd at Townsend,572.605021,MORNING
2,4932,Embarcadero at Folsom,San Francisco Caltrain (Townsend at 4th),632.021695,AFTERNOON
3,4638,San Francisco Caltrain 2 (330 Townsend),Townsend at 7th,299.492885,MORNING
4,4449,Steuart at Market,2nd at Townsend,517.991908,MORNING
5,4448,San Francisco Caltrain (Townsend at 4th),Temporary Transbay Terminal (Howard at Beale),691.246853,MORNING
6,4355,Embarcadero at Sansome,Steuart at Market,401.923307,AFTERNOON
7,4073,Temporary Transbay Terminal (Howard at Beale),San Francisco Caltrain (Townsend at 4th),656.211392,AFTERNOON
8,4001,Steuart at Market,San Francisco Caltrain (Townsend at 4th),716.052487,AFTERNOON
9,3867,Market at Sansome,2nd at South Park,356.143005,MORNING


As expected, the top ten subscriber trips are of much shorter duration than the top ten customer trips. We can also see that none of these trips begin and end at the same place. There are also several inverse trips of each other happening in the morning and in the afternoon, such as the morning trips from Harry Bridges Plaza to 2nd at Townsend in the morning and the afternoon trips from 2nd at Townsend to Harry Bridges Plaza. The most likely explanation for all of this is that these are primarily 'commuter trips' to and from one's place of work.

Let's now look at which stations have the highest utilization rates. While the exact utilization numbers may not be of utmost importance, we want to make sure to keep an eye on any particularly high or low utilization rates. Utilization rates too high can signify an increased likelihood of no bikes being avaialble for potential riders. On the other hand, utilization rates too low can signify an increased likelihood of no docks being available, meaning riders would have to venture elsewhere to end their trip. Both of the above can cause frustrating user experiences and ultimately result in lost or unrealized future trips. 

SQL Query:
  bq query --use_legacy_sql=false '
SELECT
  COUNT(*) AS LOGS,
  STATION_ID,
  AVG(bikes_available) AS AVG_BIKES_AVAIL,
  AVG(docks_available) AS AVG_DOCKS_AVAIL,
  AVG(bikes_available) / (AVG(bikes_available) + AVG(docks_available)) AS PERC_UTILIZATION
FROM
  `bigquery-public-data.san_francisco.bikeshare_status`
WHERE
  bikes_available + docks_available > 0
GROUP BY
  station_id
ORDER BY
  PERC_UTILIZATION DESC

In [25]:
! bq query --use_legacy_sql=false 'SELECT COUNT(*) AS LOGS, STATION_ID, AVG(bikes_available) AS AVG_BIKES_AVAIL, AVG(docks_available) AS AVG_DOCKS_AVAIL, AVG(bikes_available) / (AVG(bikes_available) + AVG(docks_available)) AS PERC_UTILIZATION FROM `bigquery-public-data.san_francisco.bikeshare_status` WHERE bikes_available + docks_available > 0 GROUP BY station_id ORDER BY PERC_UTILIZATION DESC'

Waiting on bqjob_r4fb47aeda89848d1_000001640fd494e6_1 ... (2s) Current status: DONE   
+---------+------------+--------------------+--------------------+---------------------+
|  LOGS   | STATION_ID |  AVG_BIKES_AVAIL   |  AVG_DOCKS_AVAIL   |  PERC_UTILIZATION   |
+---------+------------+--------------------+--------------------+---------------------+
| 1559427 |         50 | 13.335844512118827 |   9.57626679543192 |  0.5820434587241188 |
| 1559596 |         70 |  10.98917411945138 | 7.9363944252229786 |  0.5806522585311565 |
| 1559597 |         36 |  8.498530710177018 | 6.4835672292265025 |  0.5672457051442402 |
| 1559597 |         54 |  8.331310588568634 |  6.628407210324219 |  0.5569162935135863 |
| 1559569 |          3 |  8.209531607771089 |  6.779045364456499 |  0.5477192146380923 |
| 1559596 |         69 | 12.529001741476579 |  10.39665528765144 |  0.5465056781386004 |
| 1559593 |         35 |  5.980509017416742 |  5.004499250766066 |  0.5444246259457814 |
| 1559596 |         74 

We have a couple of takeaways from this table. The first is that in general, Ford is doing well in their dock station planning as most of the station utilization rates hovering right around 50%. All but one station have utilization rates between 30 and 60%. 

Stations 50 & 70 have utilization rates higher than 58%. These would be good places to start if/when we are looking to add docks to a station. (Also a note that we are assuming consistent station logging here.)

Next let's look at which zip codes most customers hail from.

bq query --use_legacy_sql=false --format=csv 'SELECT ZIP_CODE, COUNT(*) RIDES_PER_ZIP_CODE FROM `bigquery-public-data.san_francisco.bikeshare_trips` WHERE SUBSCRIBER_TYPE = "Customer" AND LENGTH(ZIP_CODE) = 5 GROUP BY ZIP_CODE, SUBSCRIBER_TYPE ORDER BY RIDES_PER_ZIP_CODE DESC LIMIT 10' > result.csv

In [85]:
! bq query --use_legacy_sql=false --format=csv 'SELECT ZIP_CODE, COUNT(*) RIDES_PER_ZIP_CODE FROM `bigquery-public-data.san_francisco.bikeshare_trips` WHERE SUBSCRIBER_TYPE = "Customer" AND LENGTH(ZIP_CODE) = 5 GROUP BY ZIP_CODE, SUBSCRIBER_TYPE ORDER BY RIDES_PER_ZIP_CODE DESC LIMIT 10' > ZC.csv

Waiting on bqjob_r2fcb90e8cfa60adf_0000016410b2dff3_1 ... (0s) Current status: DONE   

In [86]:
ZipCodes = pd.read_csv('ZC.csv')
ZipCodes


,ZIP_CODE,RIDES_PER_ZIP_CODE
0,94107,2876
1,94105,1650
2,94103,1555
3,94102,1505
4,94109,1347
5,95112,1312
6,94133,1302
7,94111,1027
8,94110,918
9,94040,827


It is important to note that these zip codes are entered by customers manually at time of sign-up and thus should be taken with a grain of salt. However, the fact that the vast majority of them come from a small subset of zip codes can be used to target our marketing efforts.

Based on the above report, we have a few recommendations for Ford GoBike which we believe will increase ridership in the San Francisco area. There are two primary ways to increase ridership: increase the general public's interest in taking one-time trips (often joy rides) and to convert customers into subscribers. 

Based on our research, we recommend offering discounts to customers who pick up and return their bikes at the same station. This will serve to promote "joy rides" around the city for visitors and others who have no interest in becoming subscribers. Given the nature of customer trips, we believe that a large majority of these are taken by travellers. Visitors to San Francisco tend to travel with others so we could offer quantity discounts for groups to entice them into biking around the city rather than walking, Segwaying, or taking the hop-on / hop-off tour buses. 

A different approach would be to add a couple docks to the stations with utilization rates higher than 55%. The goal here would be the optimize the user experience by reducing the chance that a station will either have no bikes to use or no docks available to return a bike in. Given our research, we recommend adding a couple of docks to stations 50, 70, 36, and 54. 

Lastly, we recommend increasing our marketing and advertising in the following zip codes: 94107, 94105, 94103, 94102, 94109, 95112, 94133, and 94111. These areas account for a good portion of our customers. By offering discounts or kickbacks in this area, we can likely convert some of our customers into full-time subscribers and recruit new riders through network effects.